In [1]:
import os
import itertools

In [2]:
def mkdir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)

In [3]:
# global job parameters

job_directory = f"vision_imagenet_autoaugment"
out_dir = f'{job_directory}/.out'
time_str = '00-48:00:00'
partition = 'gpu'
ntasks = 1
nodes = 1
cpu_per_gpu = 16
mem_per_cpu = 16
n_gpus = 1
# gpus_constraints = '"a100|rtx3090|v100|rtx2080ti"' # all gpus are pretty good now
gpus_constraints = '"h100|a100"' # all gpus are pretty good now
project_dir = "/home/ma2393/project/abstract_transformer/experiments/vision_imagenet"

mkdir(job_directory)
mkdir(out_dir)

In [4]:
wandb_project = 'dat-imagenet'
jobs_params = [
    # 20M scale (d = 384, L = 12)
    # dict(autoaugment=True),

    # dict(autoaugment=True, sa=6, ra=6, n_relations=6, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=1),
    # dict(autoaugment=True, sa=6, ra=6, n_relations=12, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=1),
    # dict(autoaugment=True, sa=6, ra=6, n_relations=24, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=1),
    # # dict(autoaugment=True, sa=6, ra=6, n_relations=6, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=1, n_kv_heads=3),
    # # dict(autoaugment=True, sa=6, ra=6, n_relations=12, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=1, n_kv_heads=3),
    # # dict(autoaugment=True, sa=6, ra=6, n_relations=24, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=1, n_kv_heads=3),

    # dict(autoaugment=True, sa=6, ra=6, n_relations=6, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=0, n_kv_heads=3),
    # dict(autoaugment=True, sa=6, ra=6, n_relations=12, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=0, n_kv_heads=3),
    # dict(autoaugment=True, sa=6, ra=6, n_relations=24, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=0, n_kv_heads=3),

    # 86M scale (d = 768, L = 12)
    dict(autoaugment=True, d_model=768, n_layers=12),

    dict(autoaugment=True, d_model=768, n_layers=12, sa=6, ra=6, n_relations=6, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=1),
    # dict(autoaugment=True, d_model=768, n_layers=12, sa=6, ra=6, n_relations=12, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=1),
    dict(autoaugment=True, d_model=768, n_layers=12, sa=6, ra=6, n_relations=24, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=1),

    dict(autoaugment=True, d_model=768, n_layers=12, sa=6, ra=6, n_relations=6, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=0, n_kv_heads=3),
    # dict(autoaugment=True, d_model=768, n_layers=12, sa=6, ra=6, n_relations=12, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=0, n_kv_heads=3),
    dict(autoaugment=True, d_model=768, n_layers=12, sa=6, ra=6, n_relations=24, ra_type='relational_attention', symbol_type='position_relative', symmetric_rels=0, n_kv_heads=3),
]

for j in jobs_params:
    j['pool'] = 'mean'

In [5]:
jobs_params

[{'autoaugment': True, 'd_model': 768, 'n_layers': 12, 'pool': 'mean'},
 {'autoaugment': True,
  'd_model': 768,
  'n_layers': 12,
  'sa': 6,
  'ra': 6,
  'n_relations': 6,
  'ra_type': 'relational_attention',
  'symbol_type': 'position_relative',
  'symmetric_rels': 1,
  'pool': 'mean'},
 {'autoaugment': True,
  'd_model': 768,
  'n_layers': 12,
  'sa': 6,
  'ra': 6,
  'n_relations': 24,
  'ra_type': 'relational_attention',
  'symbol_type': 'position_relative',
  'symmetric_rels': 1,
  'pool': 'mean'},
 {'autoaugment': True,
  'd_model': 768,
  'n_layers': 12,
  'sa': 6,
  'ra': 6,
  'n_relations': 6,
  'ra_type': 'relational_attention',
  'symbol_type': 'position_relative',
  'symmetric_rels': 0,
  'n_kv_heads': 3,
  'pool': 'mean'},
 {'autoaugment': True,
  'd_model': 768,
  'n_layers': 12,
  'sa': 6,
  'ra': 6,
  'n_relations': 24,
  'ra_type': 'relational_attention',
  'symbol_type': 'position_relative',
  'symmetric_rels': 0,
  'n_kv_heads': 3,
  'pool': 'mean'}]

In [6]:
len(jobs_params)

5

In [7]:
# global config parameters
n_epochs = 100
patch_size = 16
compile = True
dataset = 'imagenet'
micro_batch_size = 256
batch_size = 1024

In [8]:
# create jobs
created_jobs = []
for params in jobs_params:

    # job_name = f"vision-IMAGENET-d{params['d_model']}-sa{params['sa']}-ra{params['ra']}"
    # if "n_relations" in params:
    #     job_name += f"-nr{params['n_relations']}-L{params['n_layers']}"
    # job_name += f"-ra_type_{params['ra_type']}"
    # if "symbol_type" in params:
    #     job_name += f"-symbol_type_{params['symbol_type']}"
    # if 'activation' in params:
    #     job_name += f"-act_{params['activation']}"
    job_name = f"vision-{dataset}-"
    if params.get('autoaugment', False):
        job_name += "autoaugment"
    if "sa" in params:
        job_name += f"-sa{params['sa']}"
    if "ra" in params:
        job_name += f"-ra{params['ra']}"
    if "n_relations" in params:
        job_name += f"-nr{params['n_relations']}"
    if 'symmetric_rels' in params:
        job_name += f"-symrel{params['symmetric_rels']}"
    if "symbol_type" in params:
        job_name += f"-{params['symbol_type']}"
    if "n_kv_heads" in params:
        job_name += f"-n_kv_heads{params['n_kv_heads']}"

    job_file = os.path.join(job_directory, f"{job_name}.job")

    with open(job_file, 'w') as fh:
        fh.writelines(f"#!/bin/bash\n")
        fh.writelines(f"#SBATCH --partition={partition}\n")
        fh.writelines(f"#SBATCH --job-name={job_name}\n")
        fh.writelines(f"#SBATCH --output={out_dir}/%j-{job_name}.out\n")
        fh.writelines(f"#SBATCH --ntasks={ntasks} --nodes={nodes}\n")
        fh.writelines(f"#SBATCH --cpus-per-gpu={cpu_per_gpu}\n")
        fh.writelines(f"#SBATCH --mem-per-cpu={mem_per_cpu}G\n")
        fh.writelines(f"#SBATCH --time={time_str}\n")
        fh.writelines(f"#SBATCH --mail-type=ALL\n")
        fh.writelines(f"#SBATCH --gpus={n_gpus}\n")
        # fh.writelines(f"#SBATCH --reservation=h100\n") # NOTE: using h100 reservation for noow
        fh.writelines(f"#SBATCH -C {gpus_constraints}\n")# --gpus={n_gpus}\n")

        fh.writelines('\n')
        fh.writelines('module load StdEnv\n')
        fh.writelines('export SLURM_EXPORT_ENV=ALL\n')
        fh.writelines('\n')

        # fh.writelines(f"module restore python_env\n") # load modules i need
        fh.writelines(f"module load miniconda\n") # load modules i need
        # fh.writelines(f"conda init\n")
        fh.writelines(f"conda activate abstract_transformer\n") # activate conda environment
        fh.writelines(f"conda info --envs\n") # activate conda environment

        fh.writelines('\n')
        fh.writelines(f"nvidia-smi -L\n") # print gpu information
        fh.writelines('\n')

        fh.writelines(f"cd {project_dir}\n") # navigate to project directory
        # run python script
        fh.writelines(f"python train.py ")
        # fh.writelines(f"--dataset {params['dataset']} ")
        if params.get('autoaugment', False):
            fh.writelines(f"--autoaugment ")
        if "sa" in params:
            fh.writelines(f"--sa {params['sa']} ")
        if "ra" in params:
            fh.writelines(f"--ra {params['ra']} ")
        if "d_model" in params:
            fh.writelines(f"--d_model {params['d_model']} ")
        if "n_layers" in params:
            fh.writelines(f"--n_layers {params['n_layers']} ")
        if "n_relations" in params:
            fh.writelines(f"--n_relations {params['n_relations']} ")
        if "ra_type" in params:
            fh.writelines(f"--ra_type {params['ra_type']} ")
        if "symbol_type" in params:
            fh.writelines(f"--symbol_type {params['symbol_type']} ")
        if "symmetric_rels" in params:
            fh.writelines(f"--symmetric_rels {params['symmetric_rels']} ")
        if "n_symbols" in params:
            fh.writelines(f"--n_symbols {params['n_symbols']} ")
        if "n_kv_heads" in params:
            fh.writelines(f"--n_kv_heads {params['n_kv_heads']} ")
        if 'pool' in params:
            fh.writelines(f"--pool {params['pool']} ")
        fh.writelines(f"--patch_size {patch_size} ")
        fh.writelines(f"--batch-size {batch_size} ")
        fh.writelines(f"--micro-batch-size {micro_batch_size} ")
        if compile:
            fh.writelines(f"--compile ")

        fh.writelines(f"--max-epochs {n_epochs} ")
        fh.writelines(f"--wandb_project {wandb_project} ")

    created_jobs.append(job_file)

In [9]:
created_jobs

['vision_imagenet_autoaugment/vision-imagenet-autoaugment.job',
 'vision_imagenet_autoaugment/vision-imagenet-autoaugment-sa6-ra6-nr6-symrel1-position_relative.job',
 'vision_imagenet_autoaugment/vision-imagenet-autoaugment-sa6-ra6-nr24-symrel1-position_relative.job',
 'vision_imagenet_autoaugment/vision-imagenet-autoaugment-sa6-ra6-nr6-symrel0-position_relative-n_kv_heads3.job',
 'vision_imagenet_autoaugment/vision-imagenet-autoaugment-sa6-ra6-nr24-symrel0-position_relative-n_kv_heads3.job']

In [10]:
confirm = input("CONTINUE TO RUN ALL JOBS? (enter 'Y' or 'y')")
if confirm in ("Y", "y"):
    for job in created_jobs:
        os.system(f'sbatch {job}')
else:
    print("did not run jobs since you did not confirm.")

Submitted batch job 33629
Submitted batch job 33630
Submitted batch job 33631
Submitted batch job 33632
Submitted batch job 33633
